In [51]:
import pandas as pd
from requests_html import HTMLSession
from collections import namedtuple

In [29]:
test_url = "https://s4.goeshow.com/acrl/national/2019/profile.cfm?profile_name=session&master_key=0DFF903A-A477-DC5D-B3B4-C61346230B4A&page_key=126CB9A0-B53A-2A1E-9827-DF2938A26C80&xtemplate&userLGNKEY=0,Professional/Staff"

In [30]:
session = HTMLSession()

In [31]:
r = session.get(test_url)

We already have title, speakers, link, tag. 

We want to get the description, format, learning outcomes, type of audience

In [32]:
# find second table, 5th row
tables = r.html.find("table")
tables

[<Element 'table' class=('form-table', 'border-table', 'tb-text') width='100%'>,
 <Element 'table' width='100%' class=('form-table', 'border-table', 'tb-text')>,
 <Element 'table' >,
 <Element 'table' width='100%' class=('form-table', 'border-table', 'tb-text') id='surveyQuestionTable'>,
 <Element 'table' class=('form-table', 'noBorder-table', 'tb-text') border='0' width='100%'>,
 <Element 'table' class=('form-table', 'border-table', 'tb-text') width='100%'>]

In [33]:
tables[1].find('tr:nth-child(5)', first=True).text

'This interactive preconference will actively engage resident academic librarians by providing a format to focus on their career plans. As resident librarians, most individuals are in temporary appointments. By participating in this preconference, participants will hear from nationally recognized speakers on developing a leadership narrative. In addition, participants will work through a career planning tool to develop a road map. By participating in the preconference, resident librarians will be better prepared to pursue their next career move. Offered by the ACRL Diversity Alliance'

In [ ]:
ConfSession = namedtuple("ConfSession", ["description", "format", "audience"])

In [69]:
# takes a url 
def parse_details_page(url):
    # printing the url just for debugging purposes 
    print(url)

    session = HTMLSession()
    r = session.get(url)
    try:
        tables = r.html.find("table")
        description = tables[1].find("tr:nth-child(5)", first=True).text
    except (IndexError, AttributeError, KeyError):
        description = "NO_DESCRIPTION"

    try:
        survey_table = r.html.find("#surveyQuestionTable", first=True)
    except:
        print("There is no survey table")
        session_format = "NO_FORMAT"
        # outcomes = "NO OUTCOMES"
        audience = "NO_AUDIENCE"
    
    try:
        session_format = survey_table.find("tr:nth-child(2)", first=True).text
    except:
        session_format = "NO_FORMAT"

    # Let's skip doing learning outcomes for now rather than the extra table
    # try:
    #     outcomes = survey_table.find("tr:nth-child(2)", first=True)
    # except:
    #     outcomes = "NO OUTCOMES"
    
    try:
        audience = survey_table.find("tr:nth-child(8)", first=True).text
    except:
        audience = "NO_AUDIENCE"
    
    return ConfSession(description, session_format, audience)

In [67]:
parse_details_page(test_url)

https://s4.goeshow.com/acrl/national/2019/profile.cfm?profile_name=session&master_key=0DFF903A-A477-DC5D-B3B4-C61346230B4A&page_key=126CB9A0-B53A-2A1E-9827-DF2938A26C80&xtemplate&userLGNKEY=0,Professional/Staff


ConfSession(description='This interactive preconference will actively engage resident academic librarians by providing a format to focus on their career plans. As resident librarians, most individuals are in temporary appointments. By participating in this preconference, participants will hear from nationally recognized speakers on developing a leadership narrative. In addition, participants will work through a career planning tool to develop a road map. By participating in the preconference, resident librarians will be better prepared to pursue their next career move. Offered by the ACRL Diversity Alliance', format='Preconference, additional fee required', audience='People with some experience in the topic, but looking to grow.')

Plan:
1. import existing csv of conf data
2. get rid of all rows where there is no link
3. iterate over column of urls
4. parse each url with `parse_details_page` returning a `ConfSession`
5. compose the `ConfSession`s into a `DataFrame`
6. join the new data with the old on the vertical

In [55]:
basic_data = pd.read_csv("acrl_2019.csv")
basic_data.head()

,Unnamed: 0,title,speakers,link,tag
0,0,Taking Charge of Your Narrative (*additional f...,"Alexia Hudson-Ward, Toni Olivas, Jon Cawthorne...",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
1,1,Engaged Students are Empowered Students: Instr...,"Claire Holmes, Meg Grotti, Megan Oakleaf, Meli...",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning
2,2,Lead Your Library Towards Accessibility: Knowl...,"Stephanie Rosen, Heidi Schroeder, Andy Andrews",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
3,3,OER+ScholarlyCommunication (*additional fee re...,"Maria Bonn, Will Cross, Josh Bolick",https://s4.goeshow.com/acrl/national/2019/prof...,Scholarly Communication
4,4,"Planning, Assessing, and Communicating Library...","Lisa Stillwell, Sharon Mader",https://s4.goeshow.com/acrl/national/2019/prof...,Assessment


In [71]:
basic_data_links = basic_data[basic_data.link != "NO_LINK"]
basic_data_links

,Unnamed: 0,title,speakers,link,tag
0,0,Taking Charge of Your Narrative (*additional f...,"Alexia Hudson-Ward, Toni Olivas, Jon Cawthorne...",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
1,1,Engaged Students are Empowered Students: Instr...,"Claire Holmes, Meg Grotti, Megan Oakleaf, Meli...",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning
2,2,Lead Your Library Towards Accessibility: Knowl...,"Stephanie Rosen, Heidi Schroeder, Andy Andrews",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
3,3,OER+ScholarlyCommunication (*additional fee re...,"Maria Bonn, Will Cross, Josh Bolick",https://s4.goeshow.com/acrl/national/2019/prof...,Scholarly Communication
4,4,"Planning, Assessing, and Communicating Library...","Lisa Stillwell, Sharon Mader",https://s4.goeshow.com/acrl/national/2019/prof...,Assessment
...,...,...,...,...,...
614,614,Safe for Work: Online Professionalism Instruct...,"Christina L. Wissinger, Carmen Cole",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning
615,615,Guided Meditation,NO_SPEAKERS,https://s4.goeshow.com/acrl/national/2019/prof...,NO_TAG
616,616,Raspberry Pi a Platform for Innovation -- Solv...,Colin Nickels,https://s4.goeshow.com/acrl/national/2019/prof...,Technology and Tools
617,617,Closing Keynote: Alison Bechdel,NO_SPEAKERS,https://s4.goeshow.com/acrl/national/2019/prof...,NO_TAG


In [ ]:
conf_sessions = [parse_details_page(url) for url in basic_data_links.link]
conf_sessions

In [104]:
df = pd.DataFrame.from_records(conf_sessions, columns=ConfSession._fields)

In [92]:
# for purpose of join, add link column
df["link"] = basic_data_links.link
df.head()

,description,format,audience,link
0,This interactive preconference will actively e...,"Preconference, additional fee required","People with some experience in the topic, but ...",https://s4.goeshow.com/acrl/national/2019/prof...
1,Are your learners engaged? Are they empowered?...,"Preconference, additional fee required",People brand new to the topic.\nPeople with so...,https://s4.goeshow.com/acrl/national/2019/prof...
2,Get the tools you need to take your work and y...,"Preconference, additional fee required","People with some experience in the topic, but ...",https://s4.goeshow.com/acrl/national/2019/prof...
3,Academic libraries increasingly seek professio...,"Preconference, additional fee required",People brand new to the topic.\nPeople with ex...,https://s4.goeshow.com/acrl/national/2019/prof...
4,Libraries in higher education are increasingly...,"Preconference, additional fee required",People brand new to the topic.\nPeople with so...,https://s4.goeshow.com/acrl/national/2019/prof...


In [93]:
df.to_csv("2019_details.csv")

In [101]:
# this reset and clean makes the join easier
df2 = basic_data_links.reset_index().drop(columns=["index", "Unnamed: 0"])
df2.head()

,title,speakers,link,tag
0,Taking Charge of Your Narrative (*additional f...,"Alexia Hudson-Ward, Toni Olivas, Jon Cawthorne...",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
1,Engaged Students are Empowered Students: Instr...,"Claire Holmes, Meg Grotti, Megan Oakleaf, Meli...",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning
2,Lead Your Library Towards Accessibility: Knowl...,"Stephanie Rosen, Heidi Schroeder, Andy Andrews",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development
3,OER+ScholarlyCommunication (*additional fee re...,"Maria Bonn, Will Cross, Josh Bolick",https://s4.goeshow.com/acrl/national/2019/prof...,Scholarly Communication
4,"Planning, Assessing, and Communicating Library...","Lisa Stillwell, Sharon Mader",https://s4.goeshow.com/acrl/national/2019/prof...,Assessment


In [105]:
full_data = df2.join(df)
full_data.head()

,title,speakers,link,tag,description,format,audience
0,Taking Charge of Your Narrative (*additional f...,"Alexia Hudson-Ward, Toni Olivas, Jon Cawthorne...",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development,This interactive preconference will actively e...,"Preconference, additional fee required","People with some experience in the topic, but ..."
1,Engaged Students are Empowered Students: Instr...,"Claire Holmes, Meg Grotti, Megan Oakleaf, Meli...",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning,Are your learners engaged? Are they empowered?...,"Preconference, additional fee required",People brand new to the topic.\nPeople with so...
2,Lead Your Library Towards Accessibility: Knowl...,"Stephanie Rosen, Heidi Schroeder, Andy Andrews",https://s4.goeshow.com/acrl/national/2019/prof...,Professional/Staff Development,Get the tools you need to take your work and y...,"Preconference, additional fee required","People with some experience in the topic, but ..."
3,OER+ScholarlyCommunication (*additional fee re...,"Maria Bonn, Will Cross, Josh Bolick",https://s4.goeshow.com/acrl/national/2019/prof...,Scholarly Communication,Academic libraries increasingly seek professio...,"Preconference, additional fee required",People brand new to the topic.\nPeople with ex...
4,"Planning, Assessing, and Communicating Library...","Lisa Stillwell, Sharon Mader",https://s4.goeshow.com/acrl/national/2019/prof...,Assessment,Libraries in higher education are increasingly...,"Preconference, additional fee required",People brand new to the topic.\nPeople with so...


In [107]:
full_data.tail()

,title,speakers,link,tag,description,format,audience
282,Safe for Work: Online Professionalism Instruct...,"Christina L. Wissinger, Carmen Cole",https://s4.goeshow.com/acrl/national/2019/prof...,Teaching and Learning,Proactively managing your online information a...,Contributed Paper,People brand new to the topic.\nPeople with so...
283,Guided Meditation,NO_SPEAKERS,https://s4.goeshow.com/acrl/national/2019/prof...,NO_TAG,Mindfulness is the continuous practice of bein...,Social,NO_AUDIENCE
284,Raspberry Pi a Platform for Innovation -- Solv...,Colin Nickels,https://s4.goeshow.com/acrl/national/2019/prof...,Technology and Tools,The Raspberry Pi is small $35 wifi-compatible ...,TechConnect Presentation,People brand new to the topic.\nPeople with so...
285,Closing Keynote: Alison Bechdel,NO_SPEAKERS,https://s4.goeshow.com/acrl/national/2019/prof...,NO_TAG,Alison Bechdel is an internationally beloved c...,Keynote,NO_AUDIENCE
286,Alison Bechdel Book Signing,NO_SPEAKERS,https://s4.goeshow.com/acrl/national/2019/prof...,NO_TAG,Meet Alison Bechdel and purchase her books; Fu...,Keynote,NO_AUDIENCE


In [108]:
full_data.to_csv("2019_full_session_data.csv")